## Determine orthologous genes within bacterial genomes
### *Using OrthoDB* 
Date Created: June 26, 2025<br>
Date Modified: July 16, 2025<br>
Author: Nashira H. Ridgeway<br>

This started as an exploratory investigation, but quickly turned into a required manipulation of the OrthoDB data files for the purposes of our experiment. Essentially, OrthoDB stores orthologs in objects called "Ortho Groups" or OGs. These groups contain orthologous protein-coding genes found in other species, and can vary greatly in length. The files retrieved from OrthoDB were quite large (hundreds of millions of lines), but luckily we're focused on the essential genes in just *E. coli* and *B. subtilis* for now.

The basic workflow of this project involved pulling out genes based on the taxonomy IDs that matched *E. coli* and *B. subtilis*, finding the ortholog groups they were a part of, and isolating those for further annotation. For this preliminary stage in the project, we'll just be pulling out a few species of interest a few branches down on the phylogenetic tree, and comparing the essential genes shared between *E. coli* and *B. subtilis*. 

In [2]:
import pandas as pd
import numpy as np
import os

### Isolate the OGs of interest
Begin with the isolation of *E. coli* and *B. subtilis* genes from the OrthoDB data files

In [218]:
# Preliminary steps: determine the ID for E. coli in the OrthoDB database
species = pd.read_csv('odb12v1_species.tab', sep='\t', header=None)

In [52]:
species[species[2].str.contains('Escherichia coli')]
# Looks like orthoDB IDs are 562_0 and 83333_0 for E. coli and E. coli K12, respectively.

,0,1,2,3,4,5,6
234,562,562_0,Escherichia coli,GCF_001286085.1,NaN,NaN,NaN
4522,83333,83333_0,Escherichia coli K-12,GCF_000974885.1,NaN,NaN,NaN


In [53]:
species[species[2].str.contains('Bacillus subtilis')]

,0,1,2,3,4,5,6
526,1423,1423_0,Bacillus subtilis,GCF_008764245.1,NaN,NaN,NaN
527,1423,1423_1,Bacillus subtilis,GCF_004361725.1,NaN,NaN,NaN
528,1423,1423_2,Bacillus subtilis,GCF_002153395.1,NaN,NaN,NaN
7025,224308,224308_0,Bacillus subtilis subsp. subtilis str. 168,GCF_000009045.1,NaN,NaN,NaN
7026,224308,224308_1,Bacillus subtilis subsp. subtilis str. 168,GCF_000789275.1,NaN,NaN,NaN


In [ ]:
# Use bash to isolate the E. coli genes from the gene_xrefs table (since the files are too large to load into memory)
os.system("cat odb12v1_genes.tab | awk -F'\t' '$2 == \"562_0\"' > e_coli_genes.tab") # Isolate E. coli genes
os.system("cat odb12v1_genes.tab | awk -F'\t' '$2 == \"83333_0\"' > e_coli_k12_genes.tab") # Isolate E. coli K12 genes

2

In [ ]:
genes = pd.read_csv('e_coli_genes.tab', sep='\t', header=None)
genes.columns = ['Ortho_DB_Gene_ID', 'Ortho_DB_Species_ID', 'Protein_Sequence_ID', 'Synonyms', 'UniprotID', 'Emsembl_ID', 'NCBI_Gene_Name', 
                 'Description', 'Genomic_Coordinates', 'Genomic_DNA_ID', 'Chromosome']

In [ ]:
# Of the Ortho_DB_Gene_IDs in our tables -> isolate ones that show up in the odb12v1_gene_xrefs.tab file
os.system("cut -f1 e_coli_genes.tab > e_coli_ids.txt")
os.system("awk 'NR==FNR { ids[$1]; next } $1 in ids' e_coli_ids.txt odb12v1_gene_xrefs.tab > e_coli_xrefs.tab")

2

In [ ]:
# Do the same for E. coli K12 genes
os.system("cut -f1 e_coli_k12_genes.tab > e_coli_k12_ids.txt")
os.system("awk 'NR==FNR { ids[$1]; next } $1 in ids' e_coli_k12_ids.txt odb12v1_gene_xrefs.tab > e_coli_k12_xrefs.tab")

0

In [ ]:
# Of those gene IDs -> disseminate into OG ids 
os.system("awk 'NR==FNR { ids[$1]; next } $2 in ids' e_coli_ids.txt odb12v1_OG2genes.tab > e_coli_OG2genes.tab")
os.system("awk 'NR==FNR { ids[$1]; next } $2 in ids' e_coli_k12_ids.txt odb12v1_OG2genes.tab > e_coli_k12_OG2genes.tab")

0

In [ ]:
# Of those OG ids -> find matches in the antecedent column 
os.system("awk 'NR==FNR { ids[$1]; next } $2 in ids' e_coli_OG2genes.tab odb12v1_OG_pairs.tab > e_coli_OG_antecedent_pairs.tab")
os.system("awk 'NR==FNR { ids[$1]; next } $2 in ids' e_coli_k12_OG2genes.tab odb12v1_OG_pairs.tab > e_coli_k12_OG_antecedent_pairs.tab")

0

In [ ]:
# Do the same for the descendant column
os.system("awk 'NR==FNR { ids[$1]; next } $1 in ids' e_coli_OG2genes.tab odb12v1_OG_pairs.tab > e_coli_OG_descendent_pairs.tab")
os.system("awk 'NR==FNR { ids[$1]; next } $1 in ids' e_coli_k12_OG2genes.tab odb12v1_OG_pairs.tab > e_coli_k12_OG_descendent_pairs.tab")

0

In [ ]:
# Now we have our datasets for E. coli and E. coli K12, with antecedent and descendant OGs for their genes -- we need to disseminate the OG information to relevant genomic
# information, such as the gene, protein sequence, and species ID
# Make a list of the columns we want to translate, then use the OG2genes.tab file to get our gene IDs, then use the xrefs file to get the other IDs to make a dictionary
os.system("awk '{print $1 \"\\n\" $2}' e_coli_OG_antecedent_pairs.tab > OG_IDs_found_list.txt")
os.system("awk '{print $1 \"\\n\" $2}' e_coli_OG_descendent_pairs.tab >> OG_IDs_found_list.txt")
os.system("awk '{print $1 \"\\n\" $2}' e_coli_k12_OG_antecedent_pairs.tab >> OG_IDs_found_list.txt")
os.system("awk '{print $1 \"\\n\" $2}' e_coli_k12_OG_descendent_pairs.tab >> OG_IDs_found_list.txt")


0

In [ ]:
# Do the same preliminary filtering steps for B. subtilis (all 3 variants in OrthoDB) 1423_0, 1423_1, and 1423_2, as well as 224308_0 and 224308_1
# Use bash to isolate the E. coli genes from the gene_xrefs table (since the files are too large to load into memory)
#os.system("cat odb12v1_genes.tab | awk -F'\t' '$2 == \"1423_0\"' > subtilis_0_genes.tab") # Isolate B. subtilis 0 genes
#os.system("cat odb12v1_genes.tab | awk -F'\t' '$2 == \"1423_1\"' > subtilis_1_genes.tab") # Isolate B. subtilis 1 genes
#os.system("cat odb12v1_genes.tab | awk -F'\t' '$2 == \"1423_2\"' > subtilis_2_genes.tab") # Isolate B. subtilis 2 genes
os.system("cat odb12v1_genes.tab | awk -F'\t' '$2 == \"224308_0\"' > subtilis_168_0_genes.tab") # Isolate B. subtilis subspecies 168 0 genes
os.system("cat odb12v1_genes.tab | awk -F'\t' '$2 == \"224308_1\"' > subtilis_168_1_genes.tab") # Isolate B. subtilis subspecies 168 1 genes

# Of the Ortho_DB_Gene_IDs in our tables -> isolate ones that show up in the odb12v1_gene_xrefs.tab file
#os.system("cut -f1 subtilis_0_genes.tab > subtilis_0_ids.txt")
#os.system("awk 'NR==FNR { ids[$1]; next } $1 in ids' subtilis_0_ids.txt odb12v1_gene_xrefs.tab > subtilis_0_xrefs.tab")
#os.system("cut -f1 subtilis_1_genes.tab > subtilis_1_ids.txt")
#os.system("awk 'NR==FNR { ids[$1]; next } $1 in ids' subtilis_1_ids.txt odb12v1_gene_xrefs.tab > subtilis_1_xrefs.tab")
#os.system("cut -f1 subtilis_2_genes.tab > subtilis_2_ids.txt")
#os.system("awk 'NR==FNR { ids[$1]; next } $1 in ids' subtilis_2_ids.txt odb12v1_gene_xrefs.tab > subtilis_2_xrefs.tab")
os.system("cut -f1 subtilis_168_0_genes.tab > subtilis_168_0_ids.txt")
os.system("awk 'NR==FNR { ids[$1]; next } $1 in ids' subtilis_168_0_ids.txt odb12v1_gene_xrefs.tab > subtilis_168_0_xrefs.tab")
os.system("cut -f1 subtilis_168_1_genes.tab > subtilis_168_1_ids.txt")
os.system("awk 'NR==FNR { ids[$1]; next } $1 in ids' subtilis_168_1_ids.txt odb12v1_gene_xrefs.tab > subtilis_168_1_xrefs.tab")

# Of those gene IDs -> disseminate into OG ids 
#os.system("awk 'NR==FNR { ids[$1]; next } $2 in ids' subtilis_0_ids.txt odb12v1_OG2genes.tab > subtilis_0_OG2genes.tab")
#os.system("awk 'NR==FNR { ids[$1]; next } $2 in ids' subtilis_1_ids.txt odb12v1_OG2genes.tab > subtilis_1_OG2genes.tab")
#os.system("awk 'NR==FNR { ids[$1]; next } $2 in ids' subtilis_2_ids.txt odb12v1_OG2genes.tab > subtilis_2_OG2genes.tab")
os.system("awk 'NR==FNR { ids[$1]; next } $2 in ids' subtilis_168_0_ids.txt odb12v1_OG2genes.tab > subtilis_168_0_OG2genes.tab")
os.system("awk 'NR==FNR { ids[$1]; next } $2 in ids' subtilis_168_1_ids.txt odb12v1_OG2genes.tab > subtilis_168_1_OG2genes.tab")

# Of those OG ids -> find matches in the antecedent column 
#os.system("awk 'NR==FNR { ids[$1]; next } $2 in ids' subtilis_0_OG2genes.tab odb12v1_OG_pairs.tab > subtilis_0_OG_antecedent_pairs.tab")
#os.system("awk 'NR==FNR { ids[$1]; next } $2 in ids' subtilis_1_OG2genes.tab odb12v1_OG_pairs.tab > subtilis_1_OG_antecedent_pairs.tab")
#os.system("awk 'NR==FNR { ids[$1]; next } $2 in ids' subtilis_2_OG2genes.tab odb12v1_OG_pairs.tab > subtilis_2_OG_antecedent_pairs.tab")
os.system("awk 'NR==FNR { ids[$1]; next } $2 in ids' subtilis_168_0_OG2genes.tab odb12v1_OG_pairs.tab > subtilis_168_0_OG_antecedent_pairs.tab")
os.system("awk 'NR==FNR { ids[$1]; next } $2 in ids' subtilis_168_1_OG2genes.tab odb12v1_OG_pairs.tab > subtilis_168_1_OG_antecedent_pairs.tab")

# Do the same for the descendant column
#os.system("awk 'NR==FNR { ids[$1]; next } $1 in ids' subtilis_0_OG2genes.tab odb12v1_OG_pairs.tab > subtilis_0_OG_descendent_pairs.tab")
#os.system("awk 'NR==FNR { ids[$1]; next } $1 in ids' subtilis_1_OG2genes.tab odb12v1_OG_pairs.tab > subtilis_1_OG_descendent_pairs.tab")
#os.system("awk 'NR==FNR { ids[$1]; next } $1 in ids' subtilis_2_OG2genes.tab odb12v1_OG_pairs.tab > subtilis_2_OG_descendent_pairs.tab")
os.system("awk 'NR==FNR { ids[$1]; next } $1 in ids' subtilis_168_0_OG2genes.tab odb12v1_OG_pairs.tab > subtilis_168_0_OG_descendent_pairs.tab")
os.system("awk 'NR==FNR { ids[$1]; next } $1 in ids' subtilis_168_1_OG2genes.tab odb12v1_OG_pairs.tab > subtilis_168_1_OG_descendent_pairs.tab")

0

In [3]:
# Make a list of the columns we want to translate, then use the OG2genes.tab file to get our gene IDs, then use the xrefs file to get the other IDs to make a dictionary
#os.system("awk '{print $1 \"\\n\" $2}' subtilis_0_OG_antecedent_pairs.tab >> OG_IDs_found_list.txt")
#os.system("awk '{print $1 \"\\n\" $2}' subtilis_0_OG_descendent_pairs.tab >> OG_IDs_found_list.txt")
#os.system("awk '{print $1 \"\\n\" $2}' subtilis_1_OG_antecedent_pairs.tab >> OG_IDs_found_list.txt")
#os.system("awk '{print $1 \"\\n\" $2}' subtilis_1_OG_descendent_pairs.tab >> OG_IDs_found_list.txt")
#os.system("awk '{print $1 \"\\n\" $2}' subtilis_2_OG_antecedent_pairs.tab >> OG_IDs_found_list.txt")
#os.system("awk '{print $1 \"\\n\" $2}' subtilis_2_OG_descendent_pairs.tab >> OG_IDs_found_list.txt")

os.system("awk '{print $1 \"\\n\" $2}' subtilis_168_0_OG_antecedent_pairs.tab >> OG_IDs_found_list.txt")
os.system("awk '{print $1 \"\\n\" $2}' subtilis_168_1_OG_antecedent_pairs.tab >> OG_IDs_found_list.txt")

0

In [4]:
# Prep OG_IDs_found_list.txt for use in the next step (remove duplicates) ~250K IDs found
os.system("sort OG_IDs_found_list.txt | uniq > OG_IDs_found_list_unique.txt")

0

In [5]:
# Now of the OG IDs found, we need to trace back to the gene IDs that match them
os.system("awk 'NR==FNR { og[$1]; next } $1 in og' OG_IDs_found_list_unique.txt odb12v1_OG2genes.tab > relevant_OG_gene_pairs.txt")


0

In [11]:
# Finally, use the gene IDs to isolate the relevant genomic information (hopefully reducing the file size so we can finally import it...)
# After a few attempts, I determined the easiest way to work with this was to import the whole thing locally
# QUICKER OPTION
import csv

xref_map = {}

with open("odb12v1_genes.tab") as f:
    reader = csv.reader(f, delimiter="\t")
    for i, row in enumerate(reader, 1):
        if len(row) >= 11:
            xref_map[row[0]] = row[1:11]
        if i % 1000000 == 0:
            print(f"Read in {i:,} lines...")

errs = 0

with open("relevant_OG_gene_pairs.txt", newline="") as f_in, open("relevant_OG_gene_info_6.txt", "w", newline="") as f_out:

    reader = csv.reader(f_in, delimiter="\t")
    writer = csv.writer(f_out, delimiter="\t", lineterminator="\n")

    for j, row in enumerate(reader, 1):
        og, gene = row
        extra = xref_map.get(gene, ["NA"] * 10)
        if "NA" in extra:
            errs += 1
        writer.writerow([og, gene] + extra)
        if j % 1000000 == 0:
            print(f"Processed {j:,} lines...")
            print(f"Found {errs:,} genes with missing data in the xref_map...")


Read in 1,000,000 lines...
Read in 2,000,000 lines...
Read in 3,000,000 lines...
Read in 4,000,000 lines...
Read in 5,000,000 lines...
Read in 6,000,000 lines...
Read in 7,000,000 lines...
Read in 8,000,000 lines...
Read in 9,000,000 lines...
Read in 10,000,000 lines...
Read in 11,000,000 lines...
Read in 12,000,000 lines...
Read in 13,000,000 lines...
Read in 14,000,000 lines...
Read in 15,000,000 lines...
Read in 16,000,000 lines...
Read in 17,000,000 lines...
Read in 18,000,000 lines...
Read in 19,000,000 lines...
Read in 20,000,000 lines...
Read in 21,000,000 lines...
Read in 22,000,000 lines...
Read in 23,000,000 lines...
Read in 24,000,000 lines...
Read in 25,000,000 lines...
Read in 26,000,000 lines...
Read in 27,000,000 lines...
Read in 28,000,000 lines...
Read in 29,000,000 lines...
Read in 30,000,000 lines...
Read in 31,000,000 lines...
Read in 32,000,000 lines...
Read in 33,000,000 lines...
Read in 34,000,000 lines...
Read in 35,000,000 lines...
Read in 36,000,000 lines...
R

In [13]:
# Now utilise SQLite to import the relevant genomic information into an accessible database format, saving as xref.db
import sqlite3

conn = sqlite3.connect('xref.db')
cur = conn.cursor()
cur.execute("CREATE TABLE xref (OG_ID TEXT, gene_ID TEXT, tax_ID TEXT, wp_ID TEXT, protein_ID TEXT, synonyms TEXT, uniprot_ID TEXT, ensembl_ID TEXT, NCBI_GID TEXT, description TEXT, genomic_coordinates TEXT, genomic_DNA_ID TEXT, chromosome TEXT)")

with open("relevant_og_gene_info_6.txt") as f:
    for line in f:
        parts = line.strip().split('\t')
        if ":" in parts[1]:
            tax_id = parts[1].split(":")[0]  # extract from gene_ID
            cur.execute("INSERT INTO xref VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                        (parts[0], parts[1], tax_id, *parts[2:]))
        else:
            print("malformed line:", line.strip())
conn.commit()
conn.close()

In [16]:
## SHORTER RUNTIME VERSION -- LOAD XREF TABLE INTO MEMORY THEN MAP ORTHOLOGS TO GENES
import pickle
import pandas as pd
import numpy as np
from collections import defaultdict
import sqlite3

def extract_taxid(full_id):
    """Extract the taxonomy ID from a full OrthoDB gene ID."""
    try:
        return full_id.split(':')[0]
    except:
        print('error with splitting full_id [0]:', full_id)
        return full_id

def extract_gene_core_id(full_id):
    """Extract the core gene ID from a full OrthoDB gene ID."""
    try:
        return full_id.split(':')[1]
    except:
            print('error with splitting ful_id [1]:', full_id)
            return full_id

def map_orthologs_to_genes_shorter(bacterial_file_name, tax_id, og_to_genes):
    ortholog_map = defaultdict(list)
    counter = 0
    error_counter = []

    with open(bacterial_file_name) as f:
        for line in f:
            non_og, tax_og = line.strip().split('\t')

            tax_gene_ids = [
                gene_id for (gene_id, _) in og_to_genes.get(tax_og, [])
                if extract_taxid(gene_id) == tax_id
            ]
            if not tax_gene_ids:
                error_counter.append(tax_og)
                continue

            ortholog_ids = [
                (extract_taxid(gid), extract_gene_core_id(gid))
                for (gid, _) in og_to_genes.get(non_og, [])
                if extract_taxid(gid) != tax_id
            ]

            for tax_gene_full in tax_gene_ids:
                gene_core = extract_gene_core_id(tax_gene_full)
                ortholog_map[gene_core].extend(ortholog_ids)

            counter += 1
            if counter % 10000 == 0:
                print(f"Processed {counter} OG pairs... ({len(ortholog_map)} genes mapped)")

    return ortholog_map, error_counter


In [ ]:
conn = sqlite3.connect('xref.db')
cur = conn.cursor()

# Step 1: load xref table into memory
print("Loading xref table into memory...")
og_to_genes = defaultdict(list)
cur.execute("SELECT OG_ID, gene_ID, uniprot_ID FROM xref")
for og_id, gene_id, uniprot in cur.fetchall():
    og_to_genes[og_id].append((gene_id, uniprot))
print(f"Loaded {len(og_to_genes)} OGs from xref table.")

# Step 2: call the mapping function
e_coli_ortholog_map, e_coli_errors = map_orthologs_to_genes_shorter("e_coli_OG_antecedent_pairs.tab", "562_0", og_to_genes)

# Step 3: save results
with open("e_coli_ortholog_map.pkl", "wb") as f:
    pickle.dump(e_coli_ortholog_map, f)

print(f"Finished mapping E. coli orthologs to genes. Found {len(e_coli_ortholog_map)} unique E. coli genes with orthologs, found {len(e_coli_errors)} errors.")

# Repeat for other species

e_coli_k12_ortholog_map, e_coli_k12_errors = map_orthologs_to_genes_shorter("e_coli_k12_OG_antecedent_pairs.tab", "83333_0", og_to_genes)
with open("e_coli_k12_ortholog_map.pkl", "wb") as f:
    pickle.dump(e_coli_k12_ortholog_map, f)

print(f"Finished mapping E. coli K12 orthologs to genes. Found {len(e_coli_k12_ortholog_map)} unique E. coli K12 genes with orthologs, found {len(e_coli_k12_errors)} errors." )

b_subtilis_0_ortholog_map, b_subtilis_0_errors = map_orthologs_to_genes_shorter("subtilis_0_OG_antecedent_pairs.tab", "1423_0", og_to_genes)
with open("b_subtilis_0_ortholog_map.pkl", "wb") as f:
    pickle.dump(b_subtilis_0_ortholog_map, f)

print(f"Finished mapping B. subtilis 0 orthologs to genes. Found {len(b_subtilis_0_ortholog_map)} unique B. subtilis 0 genes with orthologs, found {len(b_subtilis_0_errors)} errors.")

b_subtilis_1_ortholog_map, b_subtilis_1_errors = map_orthologs_to_genes_shorter("subtilis_1_OG_antecedent_pairs.tab", "1423_1", og_to_genes)
with open("b_subtilis_1_ortholog_map.pkl", "wb") as f:
    pickle.dump(b_subtilis_1_ortholog_map, f) 

print(f"Finished mapping B. subtilis 1 orthologs to genes. Found {len(b_subtilis_1_ortholog_map)} unique B. subtilis 1 genes with orthologs, found {len(b_subtilis_1_errors)} errors.")

b_subtilis_2_ortholog_map, b_subtilis_2_errors = map_orthologs_to_genes_shorter("subtilis_2_OG_antecedent_pairs.tab", "1423_2", og_to_genes)
with open("b_subtilis_2_ortholog_map.pkl", "wb") as f:
    pickle.dump(b_subtilis_2_ortholog_map, f)

print(f"Finished mapping B. subtilis 2 orthologs to genes. Found {len(b_subtilis_2_ortholog_map)} unique B. subtilis 2 genes with orthologs, found {len(b_subtilis_2_errors)} errors.")  

b_subtilis_168_0_ortholog_map, b_subtilis_168_0_errors = map_orthologs_to_genes_shorter("subtilis_168_0_OG_antecedent_pairs.tab", "224308_0", og_to_genes)
with open("b_subtilis_168_0_ortholog_map.pkl", "wb") as f:
    pickle.dump(b_subtilis_168_0_ortholog_map, f)

print(f"Finished mapping B. subtilis 168_0 orthologs to genes. Found {len(b_subtilis_168_0_ortholog_map)} unique B. subtilis 168_0 genes with orthologs, found {len(b_subtilis_168_0_errors)} errors.")  

b_subtilis_168_1_ortholog_map, b_subtilis_168_1_errors = map_orthologs_to_genes_shorter("subtilis_168_1_OG_antecedent_pairs.tab", "224308_1", og_to_genes)
with open("b_subtilis_168_1_ortholog_map.pkl", "wb") as f:
    pickle.dump(b_subtilis_168_1_ortholog_map, f)

print(f"Finished mapping B. subtilis 168_1 orthologs to genes. Found {len(b_subtilis_168_1_ortholog_map)} unique B. subtilis 168_1 genes with orthologs, found {len(b_subtilis_168_1_errors)} errors.")  


conn.close()

#  Created .gpickle DB files for *E. coli* and *B. subtilis*
Now we have .gpickle files with the databases containing the OGs and genes associated with our organisms of interest. We may go on and begin isolating the essential genes from here!


,0,1,2,3,4,5,6
6761,208964,208964_0,Pseudomonas aeruginosa PAO1,GCF_000006765.1,NaN,NaN,NaN
